# Inicializar bibliotecas

In [2]:
!wget https://emanuel3queijos.github.io/AWS-University-ETL/python_scripts/requirements.txt -O requirements.txt
!sed -i '/pywin32/d' requirements.txt
!pip install -r requirements.txt


--2024-11-09 23:39:50--  https://emanuel3queijos.github.io/AWS-University-ETL/python_scripts/requirements.txt
Resolving emanuel3queijos.github.io (emanuel3queijos.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to emanuel3queijos.github.io (emanuel3queijos.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 691 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     691  --.-KB/s    in 0s      

2024-11-09 23:39:51 (32.8 MB/s) - ‘requirements.txt’ saved [691/691]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.

# Scripts para adicionar data dentro do banco



In [3]:
import psycopg2

# Função para criar a conexão
def create_connection():
    try:
        conn = psycopg2.connect(
            host="db-manu-etluniversityproject-instance.c44vb5orqds7.us-east-1.rds.amazonaws.com",
            dbname="AWS-University-ETL-DB",
            user="postgres",
            password="Emanu0710**",
            port=5432
        )

        return conn
    except Exception as error:
        print(f"Erro ao conectar ao banco de dados: {error}")
        return None
create_connection()

<connection object at 0x79f0a8ec3380; dsn: 'user=postgres password=xxx dbname=AWS-University-ETL-DB host=db-manu-etluniversityproject-instance.c44vb5orqds7.us-east-1.rds.amazonaws.com port=5432', closed: 0>

In [5]:
def inserir_departamentos():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            # Inserir departamentos
            departamentos = [
                ("Escola de Tecnologia", "Departamento focado em áreas tecnológicas."),
                ("Escola de Engenharia", "Departamento voltado para as engenharias."),
                ("Escola de Saúde", "Departamento voltado para ciências da saúde."),
                ("Ciências Biológicas", "Departamento voltado para biologia e afins.")
            ]
            for nome, descricao in departamentos:
                cur.execute("""
                    INSERT INTO tb_departamento (nome_departamento, descricao)
                    VALUES (%s, %s)
                    RETURNING cod_departamento
                """, (nome, descricao))
                cod_departamento = cur.fetchone()[0]
                print(f"Departamento {nome} inserido com sucesso. Código: {cod_departamento}")

            conn.commit()
            print("Departamentos e inseridos com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

# Executar o script
inserir_departamentos()


Departamento Escola de Tecnologia inserido com sucesso. Código: 10
Departamento Escola de Engenharia inserido com sucesso. Código: 11
Departamento Escola de Saúde inserido com sucesso. Código: 12
Departamento Ciências Biológicas inserido com sucesso. Código: 13
Departamentos e inseridos com sucesso.


In [ ]:
def inserir_cursos():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            # Dicionário onde as chaves são os nomes dos departamentos
            cursos_por_departamento = {
                "Escola de Tecnologia": [
                    ("Engenharia de Software", "Curso voltado ao desenvolvimento de software.", 4, 1200.00),
                    ("Ciência da Computação", "Foco em computação e sistemas.", 4, 1150.00),
                    ("Inteligência Artificial", "Curso para aprendizado de IA.", 4, 1300.00),
                    ("Redes de Computadores", "Infraestrutura e comunicação de redes.", 3, 1100.00)
                ],
                "Escola de Engenharia": [
                    ("Engenharia Civil", "Foco em obras e infraestrutura.", 5, 1400.00),
                    ("Engenharia Elétrica", "Curso de sistemas elétricos.", 5, 1350.00),
                    ("Engenharia Mecânica", "Curso sobre mecânica aplicada.", 5, 1380.00),
                    ("Engenharia de Controle e Automação", "Automação industrial.", 5, 1450.00)
                ],
                "Escola de Saúde": [
                    ("Medicina", "Foco em prática médica.", 6, 2000.00),
                    ("Enfermagem", "Prática de enfermagem.", 4, 1100.00),
                    ("Farmácia", "Estudos sobre medicamentos.", 4, 1250.00),
                    ("Biomedicina", "Ciência biomédica.", 4, 1200.00)
                ],
                "Ciências Biológicas": [
                    ("Biologia", "Estudo da vida.", 4, 1100.00),
                    ("Ecologia", "Curso sobre ecossistemas.", 4, 1050.00),
                    ("Zoologia", "Estudo dos animais.", 4, 1100.00),
                    ("Botânica", "Estudo das plantas.", 4, 1000.00)
                ]
            }

            for nome_departamento, cursos in cursos_por_departamento.items():
                # Busca o código do departamento pelo nome
                cur.execute("""
                    SELECT cod_departamento FROM tb_departamento WHERE nome_departamento = %s
                """, (nome_departamento,))
                cod_departamento = cur.fetchone()

                if cod_departamento is None:
                    print(f"Departamento '{nome_departamento}' não encontrado.")
                    continue

                cod_departamento = cod_departamento[0]

                for nome_curso, descricao_curso, duracao, mensalidade in cursos:
                    cur.execute("""
                        INSERT INTO tb_curso (nome_curso, descricao, cod_departamento, duracao, valor_mensalidade)
                        VALUES (%s, %s, %s, %s, %s)
                        RETURNING cod_curso
                    """, (nome_curso, descricao_curso, cod_departamento, duracao, mensalidade))
                    cod_curso = cur.fetchone()[0]

                    print(f"Curso {nome_curso} inserido com sucesso. Código: {cod_curso}")

            conn.commit()
            print("Departamentos, cursos e disciplinas inseridos com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

# Executar o script
inserir_cursos()


In [11]:
def inserir_disciplinas():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            # Dados de disciplinas expandido por departamento e curso, cobrindo 8 semestres
            disciplinas_por_departamento = {
                "Escola de Tecnologia": {
                    "Ciência da Computação": [
                        ("Introdução à Computação", "Fundamentos básicos de computação.", 60, 1),
                        ("Programação I", "Lógica de programação e algoritmos.", 60, 1),
                        ("Estrutura de Dados", "Curso sobre estruturas de dados.", 60, 2),
                        ("Sistemas Operacionais", "Princípios dos sistemas operacionais.", 60, 2),
                        ("Algoritmos Avançados", "Estratégias avançadas de algoritmos.", 60, 3),
                        ("Banco de Dados", "Introdução a banco de dados.", 60, 3),
                        ("Redes de Computadores", "Conceitos e protocolos de redes.", 60, 4),
                        ("Engenharia de Software", "Princípios da engenharia de software.", 80, 4),
                        ("Inteligência Artificial", "Fundamentos de IA.", 80, 5),
                        ("Computação Gráfica", "Processamento gráfico em computação.", 80, 5),
                        ("Machine Learning", "Conceitos e aplicações de aprendizado de máquina.", 80, 6),
                        ("Computação em Nuvem", "Fundamentos da computação em nuvem.", 80, 6),
                        ("Segurança da Informação", "Proteção e privacidade de dados.", 60, 7),
                        ("Internet das Coisas", "Tecnologias para IoT.", 60, 7),
                        ("Projeto de Conclusão de Curso I", "Projeto inicial para TCC.", 80, 8),
                        ("Projeto de Conclusão de Curso II", "Desenvolvimento final do TCC.", 80, 8)
                    ],
                    "Engenharia de Software": [
                        ("Lógica de Programação", "Foco em lógica e algoritmos.", 60, 1),
                        ("Estrutura de Dados", "Curso sobre estruturas de dados.", 80, 1),
                        ("Banco de Dados I", "Conceitos básicos de banco de dados.", 60, 2),
                        ("Engenharia de Requisitos", "Práticas de levantamento de requisitos.", 60, 3),
                        ("Arquitetura de Software", "Conceitos de arquitetura e design.", 60, 4),
                        ("Gestão de Projetos", "Introdução ao gerenciamento de projetos.", 60, 5),
                        ("Teste de Software", "Processos e técnicas de testes.", 60, 6),
                        ("DevOps", "Integração contínua e entrega de software.", 60, 7),
                        ("Gestão Ágil", "Práticas de metodologias ágeis.", 60, 7),
                        ("Projeto de Conclusão de Curso", "Desenvolvimento do TCC.", 80, 8)
                    ]
                },
                "Escola de Engenharia": {
                    "Engenharia Civil": [
                        ("Cálculo Diferencial e Integral I", "Fundamentos de cálculo.", 80, 1),
                        ("Materiais de Construção", "Estudo de materiais de construção.", 80, 2),
                        ("Topografia", "Métodos de levantamento topográfico.", 60, 2),
                        ("Resistência dos Materiais", "Estudo de resistência estrutural.", 60, 3),
                        ("Mecânica dos Solos", "Estudo do solo para construção.", 60, 4),
                        ("Hidráulica", "Princípios da hidráulica aplicada.", 60, 5),
                        ("Estruturas de Concreto", "Projeto de estruturas de concreto.", 80, 6),
                        ("Estradas e Pavimentação", "Planejamento e construção de estradas.", 60, 7),
                        ("Planejamento Urbano", "Conceitos e técnicas de planejamento urbano.", 60, 8)
                    ],
                    "Engenharia Elétrica": [
                        ("Fundamentos de Eletricidade", "Princípios básicos da eletricidade.", 60, 1),
                        ("Circuitos Elétricos", "Fundamentos de circuitos.", 70, 2),
                        ("Eletrônica Digital", "Aplicações digitais.", 65, 3),
                        ("Máquinas Elétricas", "Funcionamento de máquinas elétricas.", 70, 4),
                        ("Sistemas de Potência", "Análise de sistemas elétricos de potência.", 70, 5),
                        ("Automação Industrial", "Fundamentos da automação.", 60, 6),
                        ("Qualidade da Energia", "Aspectos de qualidade e estabilidade.", 60, 7),
                        ("Fontes Renováveis de Energia", "Sistemas de energia sustentável.", 60, 8)
                    ]
                },
                "Escola de Saúde": {
                    "Medicina": [
                        ("Anatomia Humana I", "Estudo do corpo humano.", 100, 1),
                        ("Fisiologia", "Estudo das funções do corpo.", 100, 2),
                        ("Patologia", "Estudo das doenças.", 100, 3),
                        ("Farmacologia", "Estudo de medicamentos.", 90, 4),
                        ("Microbiologia", "Estudo dos microrganismos.", 80, 5),
                        ("Clínica Médica", "Práticas de atendimento clínico.", 100, 6),
                        ("Psiquiatria", "Estudo das doenças mentais.", 90, 7),
                        ("Estágio Supervisionado", "Prática em ambiente clínico.", 100, 8)
                    ],
                    "Enfermagem": [
                        ("Fundamentos da Enfermagem", "Conceitos básicos da enfermagem.", 80, 1),
                        ("Cuidados Básicos", "Práticas de cuidados em saúde.", 80, 2),
                        ("Anatomia Aplicada", "Estudo da anatomia na prática de enfermagem.", 60, 3),
                        ("Enfermagem Pediátrica", "Cuidados com crianças.", 60, 4),
                        ("Saúde da Mulher", "Cuidados com a saúde feminina.", 60, 5),
                        ("Enfermagem Psiquiátrica", "Apoio e cuidados psiquiátricos.", 60, 6),
                        ("Gestão em Enfermagem", "Administração em ambientes hospitalares.", 60, 7),
                        ("Estágio em Enfermagem", "Prática supervisionada.", 80, 8)
                    ]
                }
            }

            for nome_departamento, cursos in disciplinas_por_departamento.items():
                cur.execute("SELECT cod_departamento FROM tb_departamento WHERE nome_departamento = %s", (nome_departamento,))
                cod_departamento = cur.fetchone()

                if cod_departamento is None:
                    print(f"Departamento '{nome_departamento}' não encontrado.")
                    continue
                cod_departamento = cod_departamento[0]

                for nome_curso, disciplinas in cursos.items():
                    cur.execute("SELECT cod_curso FROM tb_curso WHERE nome_curso = %s AND cod_departamento = %s", (nome_curso, cod_departamento))
                    cod_curso = cur.fetchone()

                    if cod_curso is None:
                        print(f"Curso '{nome_curso}' no departamento '{nome_departamento}' não encontrado.")
                        continue
                    cod_curso = cod_curso[0]

                    for nome_disciplina, descricao_disciplina, carga_horaria, semestre_oferecido in disciplinas:
                        cur.execute("""
                            INSERT INTO tb_disciplina (nome_disciplinas, descricao, carga_horaria, semestre_oferecido, cod_departamento)
                            VALUES (%s, %s, %s, %s, %s)
                            RETURNING cod_disciplina
                        """, (nome_disciplina, descricao_disciplina, carga_horaria, semestre_oferecido, cod_departamento))
                        cod_disciplina = cur.fetchone()[0]

                        cur.execute("""
                            INSERT INTO tb_curso_disciplina (cod_departamento, cod_curso, cod_disciplina)
                            VALUES (%s, %s, %s)
                        """, (cod_departamento, cod_curso, cod_disciplina))

                        print(f"Disciplina '{nome_disciplina}' inserida com sucesso no curso '{nome_curso}'. Código: {cod_disciplina}")

            conn.commit()
            print("Disciplinas inseridas com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

# Executar o script
inserir_disciplinas()


Disciplina 'Introdução à Computação' inserida com sucesso no curso 'Ciência da Computação'. Código: 15
Disciplina 'Programação I' inserida com sucesso no curso 'Ciência da Computação'. Código: 16
Disciplina 'Estrutura de Dados' inserida com sucesso no curso 'Ciência da Computação'. Código: 17
Disciplina 'Sistemas Operacionais' inserida com sucesso no curso 'Ciência da Computação'. Código: 18
Disciplina 'Algoritmos Avançados' inserida com sucesso no curso 'Ciência da Computação'. Código: 19
Disciplina 'Banco de Dados' inserida com sucesso no curso 'Ciência da Computação'. Código: 20
Disciplina 'Redes de Computadores' inserida com sucesso no curso 'Ciência da Computação'. Código: 21
Disciplina 'Engenharia de Software' inserida com sucesso no curso 'Ciência da Computação'. Código: 22
Disciplina 'Inteligência Artificial' inserida com sucesso no curso 'Ciência da Computação'. Código: 23
Disciplina 'Computação Gráfica' inserida com sucesso no curso 'Ciência da Computação'. Código: 24
Discipl

# operações extras(deleta tudo não vei 😭)

In [13]:
from faker import Faker
import random

fake = Faker()

def inserir_professores():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            # Consultar disciplinas e departamentos para alocar professores
            cur.execute("SELECT cod_disciplina, cod_departamento FROM tb_disciplina")
            disciplinas = cur.fetchall()

            # Dicionário para armazenar IDs de professores já associados às disciplinas
            professor_ids = {}

            for cod_disciplina, cod_departamento in disciplinas:
                for _ in range(2):  # Dois professores por disciplina
                    # Gerar dados fictícios para o professor
                    nome = fake.name()
                    data_nascimento = fake.date_of_birth(minimum_age=30, maximum_age=60)
                    email = fake.email()

                    # Inserir professor no departamento
                    cur.execute("""
                        INSERT INTO tb_professor (nome, data_nascimento, email, cod_departamento)
                        VALUES (%s, %s, %s, %s)
                        RETURNING cod_professor
                    """, (nome, data_nascimento, email, cod_departamento))
                    cod_professor = cur.fetchone()[0]

                    # Inserir associação do professor com a disciplina
                    cur.execute("""
                        INSERT INTO prof_disciplina (cod_professor, cod_disciplina)
                        VALUES (%s, %s)
                    """, (cod_professor, cod_disciplina))

                    # Armazenar ID do professor para futuras consultas
                    professor_ids[cod_professor] = (nome, email)

                    print(f"Professor '{nome}' alocado na disciplina de código '{cod_disciplina}'.")

            conn.commit()
            print("Professores inseridos e alocados com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir professores: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

# Executar o script
inserir_professores()


ModuleNotFoundError: No module named 'faker'

In [ ]:

def export_to_csv():
    conn = create_connection()
    try:
        cur = conn.cursor()

        tables = ["tb_aluno", "tb_curso", "tb_disciplina", "tb_turma", "tb_matricula", "tb_inscricao", "tb_turma_aluno"]
        for table in tables:
            with open(f'{table}.csv', 'w') as f:
                cur.copy_expert(f"COPY {table} TO STDOUT WITH CSV HEADER", f)

        cur.close()
        print("Dados exportados com sucesso.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao exportar dados: {error}")
    finally:
        if conn is not None:
            conn.close()


#Export tables
export_to_csv()

In [ ]:
# Limpar as tabelas
def clean_tables(conn):
    conn = create_connection()
    try:
        cur = conn.cursor()
        truncate_sql = """TRUNCATE TABLE tb_turma_aluno, tb_inscricao, tb_matricula, tb_turma, tb_disciplina, tb_curso, tb_aluno RESTART IDENTITY CASCADE;"""
        cur.execute(truncate_sql)
        conn.commit()
        print("Tabelas limpas com sucesso")
        cur.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao truncar tabelas: {error}")

    finally:
        if conn is not None:
            conn.close()

clean_tables()

Tabelas limpas com sucesso
